In [3]:
with open('./examen5.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
    contains = " ".join(lines)

In [4]:
import re
#list dans question avec comme separateur Question XX
questions =  re.split ("Question \d{,2}",contains)[1:]

In [5]:
print(questions[3])

 Incorrect Amazon EC2 Auto Scaling needs to terminate an instance from Availability Zone (AZ) us-east-1a as it has the most number of instances amongst the Availability Zone (AZs) being used currently. There are 4 instances in the Availability Zone (AZ) us-east-1a like so: Instance A has the oldest launch template, Instance B has the oldest launch configuration, Instance C has the newest launch configuration and Instance D is closest to the next billing hour.  Which of the following instances would be terminated per the default termination policy?  Instance C  Bonne réponse Instance B  Votre réponse est incorrecte Instance A  Instance D  Explication générale Correct option:  Instance B  Per the default termination policy, the first priority is given to any allocation strategy for On-Demand vs Spot instances. As no such information has been provided for the given use-case, so this criterion can be ignored. The next priority is to consider any instance with the oldest launch template unl

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage

/Users/jonathanbizet/Documents/question_type_examen/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [9]:
import os
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [10]:
OPENAI_API_KEY

'sk-7rxHqoSj5qyPTGb-8T50xTfTXRwp1p34bqLcSSd8MAT3BlbkFJBNpWwAYxPyEDjKFZGdlC2Y9b9uBGDKozPZoJKpKFcA'

In [11]:
chat_model = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

/Users/jonathanbizet/Documents/question_type_examen/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
def get_langchain_response(prompt: str,textquestion:str) -> str:
        messages = [SystemMessage(content=prompt),HumanMessage(content=textquestion)]
        response = chat_model(messages)
        return response.content

In [13]:
from pydantic import BaseModel, Field, model_validator
from typing import List, Optional

In [14]:
class Question(BaseModel):
    question: str
    options: List[str]
    answer:  List[str]
    theme: Optional[str] = Field(default='')
    img_path: Optional[str] = Field(default=None)

    @model_validator(mode='after')
    def correct_must_be_in_responses(self):
        for a in self.answer:
            if a not in self.options:
                raise ValueError('The correct answer must be in the options list') 
        return self

In [15]:
import json

In [16]:
def parse_question(text: str) -> Question:
    prompt = """
    I extract the data from this question:
     Correct A company runs a shopping application that uses Amazon DynamoDB to store customer information. In case of data corruption, a solutions architect needs to design a solution that meets a recovery point objective (RPO) of 15 minutes and a recovery time objective (RTO) of 1 hour.  What should the solutions architect recommend to meet these requirements?  Votre réponse est correcte B. Configure DynamoDB point-in-time recovery. For RPO recovery, restore to the desired point in time.  D. Schedule Amazon Elastic Block Store (Amazon EBS) snapshots for the DynamoDB table every 15 minutes. For RPO recovery, restore the DynamoDB table by using the EBS snapshot.  C. Export the DynamoDB data to Amazon S3 Glacier on a daily basis. For RPO recovery, import the data from S3 Glacier to DynamoDB.  A. Configure DynamoDB global tables. For RPO recovery, point the application to a different AWS Region.  Explication générale Option B is correct.  Point-in-time recovery helps protect your DynamoDB tables from accidental write or delete operations. With point-in-time recovery, you don’t have to worry about creating, maintaining, or scheduling on-demand backups.  With point-in-time recovery, you can restore that table to any point in time during the last 35 days. DynamoDB maintains incremental backups of your table.  Point-in-time recovery for DynamoDB  Protect your DynamoDB tables from accidental write or delete operations with point-in-time recovery.  docs.aws.amazon.com  Arguments about others:  The correct answer is B. Configure DynamoDB point-in-time recovery. For RPO recovery, restore to the desired point in time.  Here’s why:  A. Configuring DynamoDB global tables would provide high availability and replication of data across multiple AWS Regions, but it doesn’t directly address data corruption or the specified RPO and RTO requirements.  C. Exporting DynamoDB data to Amazon S3 Glacier on a daily basis doesn’t meet the RPO of 15 minutes, and restoring data from Glacier to DynamoDB can take a considerable amount of time, making it unsuitable for an RTO of 1 hour.  D. Scheduling Amazon Elastic Block Store (Amazon EBS) snapshots for the DynamoDB table every 15 minutes is not a valid approach for DynamoDB, as DynamoDB is a managed NoSQL database service, and EBS snapshots are not applicable for it.  Domaine Design Resilient Architecture 
    #######
    {
        "question": "A company runs a shopping application that uses Amazon DynamoDB to store customer information. In case of data corruption, a solutions architect needs to design a solution that meets a recovery point objective (RPO) of 15 minutes and a recovery time objective (RTO) of 1 hour.

                    What should the solutions architect recommend to meet these requirements?",
        "options": [
            "Configure DynamoDB point-in-time recovery. For RPO recovery, restore to the desired point in time.",
            "Schedule Amazon Elastic Block Store (Amazon EBS) snapshots for the DynamoDB table every 15 minutes. For RPO recovery, restore the DynamoDB table by using the EBS snapshot.",
            "Export the DynamoDB data to Amazon S3 Glacier on a daily basis. For RPO recovery, import the data from S3 Glacier to DynamoDB.",
            "Configure DynamoDB global tables. For RPO recovery, point the application to a different AWS Region."
        ],
        "answer": ["Configure DynamoDB point-in-time recovery. For RPO recovery, restore to the desired point in time."]
    }
    other exemple:
     Incorrect A company has an application that generates a large number of files, each approximately 5 MB in size. The files are stored in Amazon S3. Company policy requires the files to be stored for 4 years before they can be deleted. Immediate accessibility is always required as the files contain critical business data that is not easy to reproduce. The files are frequently accessed in the first 30 days of the object creation but are rarely accessed after the first 30 days.  Which storage solution is MOST cost-effective?  A. Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Glacier 30 days from object creation. Delete the files 4 years after object creation.  Votre réponse est incorrecte B. Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 One Zone-Infrequent Access (S3 One Zone-IA) 30 days from object creation. Delete the files 4 years after object creation.  Bonne réponse C. Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Standard-Infrequent Access (S3 Standard-IA) 30 days from object creation. Delete the files 4 years after object creation.  D. Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Standard-Infrequent Access (S3 Standard-IA) 30 days from object creation. Move the files to S3 Glacier 4 years after object creation.  Explication générale Option C is correct.  The files are frequently accessed in the first 30 days, and S3 Standard provides immediate accessibility. Therefore, you should keep the files in S3 Standard for the initial 30 days to ensure quick access.After the initial 30 days, the files are rarely accessed. Transitioning them to S3 Standard-IA is cost-effective because it offers lower storage costs than S3 Standard while still providing quick access when needed.  Arguments about others:  Options A and B involve transitioning the files to S3 Glacier or S3 One Zone-IA, respectively, after 30 days. While these options reduce storage costs compared to S3 Standard, they may introduce retrieval delays and costs if the files are needed during the 4-year retention period.  Option D moves the files to S3 Standard-IA initially but then transitions them to S3 Glacier after 4 years. This introduces additional complexity without a clear benefit in terms of cost savings for your use case, as it doesn’t take into account the initial 30-day access requirement.  Domaine Design Cost-Optimised Architecture 

    #######
    '{"question":"A company has an application that generates a large number of files, each approximately 5 MB in size. The files are stored in Amazon S3. Company policy requires the files to be stored for 4 years before they can be deleted. Immediate accessibility is always required as the files contain critical business data that is not easy to reproduce. The files are frequently accessed in the first 30 days of the object creation but are rarely accessed after the first 30 days.

Which storage solution is MOST cost-effective?","options":["Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Glacier 30 days from object creation. Delete the files 4 years after object creation.","Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 One Zone-Infrequent Access (S3 One Zone-IA) 30 days from object creation. Delete the files 4 years after object creation.","Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Standard-Infrequent Access (S3 Standard-IA) 30 days from object creation. Delete the files 4 years after object creation.","Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Standard-Infrequent Access (S3 Standard-IA) 30 days from object creation. Move the files to S3 Glacier 4 years after object creation."],"answer":["Create an S3 bucket lifecycle policy to move files from S3 Standard to S3 Standard-Infrequent Access (S3 Standard-IA) 30 days from object creation. Delete the files 4 years after object creation."]}'
    ######
    other exemple:
    ' Incorrect Which of the following are characteristics of the Auto Scaling service on AWS? (choose 3)  D. Sends traffic to healthy instances.  Votre sélection est incorrecte F. Collects and tracks metrics and sets alarms.  A. Delivers push notifications.  Sélection correcte B. Launches instances from a specified Amazon Machine Image (AMI).  Votre sélection est correcte C. Enforces a minimum number of running Amazon EC2 instances.  Votre sélection est correcte E. Responds to changing conditions by adding or terminating Amazon EC2 instances.  Explication générale Option B,C and E are correct.  Domaine Design High Performing Architecture '
    {
  "question": "Which of the following are characteristics of the Auto Scaling service on AWS? (choose 3)",
  "options": [
    "Delivers push notifications.",
    "Launches instances from a specified Amazon Machine Image (AMI).",
    "Enforces a minimum number of running Amazon EC2 instances.",
    "Sends traffic to healthy instances.",
    "Responds to changing conditions by adding or terminating Amazon EC2 instances.",
    "Collects and tracks metrics and sets alarms."
  ],
  "answer": [
    "Launches instances from a specified Amazon Machine Image (AMI)",
    "Enforces a minimum number of running Amazon EC2 instances.",
    "Responds to changing conditions by adding or terminating Amazon EC2 instances."
  ]}
    In some questions, it is asked to find the "incorrect" answer or "Except" and the data should be put in the "answer" field
    answer is a list
    Be careful that the "answer" must be in list "options"
    Correct any typos while respecting the previous constraints
    Remove unnecessary line breaks
    please base only on the text in order avoid halucination

    Respond without an introductory phrase like "Here is the JSON data corresponding to the question
    """
    response = get_langchain_response(prompt=prompt,textquestion=text)
    question_data = json.loads(response)
    question = Question(**question_data)
    return question


In [17]:
questions[27]

' Incorrect An e-commerce application uses an Amazon Aurora Multi-AZ deployment for its database. While analyzing the performance metrics, the engineering team has found that the database reads are causing high input/output (I/O) and adding latency to the write requests against the database.  As an AWS Certified Solutions Architect Associate, what would you recommend to separate the read requests from the write requests?  Activate read-through caching on the Amazon Aurora database  Provision another Amazon Aurora database and link it to the primary database as a read replica  Votre réponse est incorrecte Configure the application to read from the Multi-AZ standby instance  Bonne réponse Set up a read replica and modify the application to use the appropriate endpoint  Explication générale Correct option:  Set up a read replica and modify the application to use the appropriate endpoint  An Amazon Aurora DB cluster consists of one or more DB instances and a cluster volume that manages the

In [18]:
question = map(parse_question,questions)

In [19]:
parsed_question = list()

In [20]:

for i,question in enumerate(questions):
    print(i)
    try:
        q = parse_question(question)
        parsed_question.append(q)
    except Exception as e:
        print(e)
    

0


/Users/jonathanbizet/Documents/question_type_examen/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64


In [21]:
from io import StringIO

In [22]:
data_question_list=list()
for pq in parsed_question:
   data_question_list.append(pq.dict())

In [23]:
with open('examen5.json', 'w', encoding='utf-8') as f:
    json.dump(data_question_list, f, ensure_ascii=False, indent=4)